In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('E:/College/Analytics/Python/database.sqlite')

In [3]:
#sqlite_master is a table with database schema
pd.read_sql(""" SELECT *
                FROM sqlite_master
                WHERE type='table';""",
           conn)

,type,name,tbl_name,rootpage,sql
0,table,station,station,2,CREATE TABLE station (\n id INTEGER PRIMARY...
1,table,status,status,3,"CREATE TABLE status (\n station_id INTEGER,..."
2,table,trip,trip,4,CREATE TABLE trip (\n id INTEGER PRIMARY KE...
3,table,weather,weather,5,"CREATE TABLE weather (\n date TEXT,\n ma..."


In [4]:
pd.read_sql('''SELECT *  FROM station LIMIT 10; ''', con=conn)

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
5,7,Paseo de San Antonio,37.333798,-121.886943,15,San Jose,8/7/2013
6,8,San Salvador at 1st,37.330165,-121.885831,15,San Jose,8/5/2013
7,9,Japantown,37.348742,-121.894715,15,San Jose,8/5/2013
8,10,San Jose City Hall,37.337391,-121.886995,15,San Jose,8/6/2013
9,11,MLK Library,37.335885,-121.885660,19,San Jose,8/6/2013


In [5]:
# Total docks and station with average capacity of each City
pd.read_sql(''' SELECT city, 
    SUM(dock_count) AS Total_capacity, 
    COUNT(name) AS Station_count, 
    ROUND(SUM(dock_count)/COUNT(name), 2) AS Average_capacity
    FROM station
    GROUP BY city
    ORDER BY station_count DESC;
''', con=conn)

,city,Total_capacity,Station_count,Average_capacity
0,San Francisco,665,35,19.0
1,San Jose,264,16,16.0
2,Mountain View,117,7,16.0
3,Redwood City,115,7,16.0
4,Palo Alto,75,5,15.0


In [6]:
# Docks and Station count per city and date
pd.read_sql('''
    SELECT city,
           CASE
           -- m/d/yyyy
           WHEN (length(installation_date) = 8 AND substr(installation_date,2,1) = '/') 
           THEN substr(installation_date,5,4)||'-0'||substr(installation_date,1,1)||'-0'||substr(installation_date,3,1)
           -- m/dd/yyyy
           WHEN (length(installation_date) = 9 AND substr(installation_date,2,1) = '/') 
           THEN substr(installation_date,6,4)||'-0'||substr(installation_date,1,1)||'-'||substr(installation_date,3,2)
           -- mm/d/yyyy
           WHEN (length(installation_date) = 9 AND substr(installation_date,3,1) = '/') 
           THEN substr(installation_date,6,4)||'-0'||substr(installation_date,1,2)||'-'||substr(installation_date,4,1)
           -- mm/dd/yyyy
           WHEN (length(installation_date) = 10 AND substr(installation_date,3,1) = '/') 
           THEN substr(installation_date,7,4)||'-'||substr(installation_date,1,2)||'-'||substr(installation_date,4,2)
           ELSE installation_date
           END AS installed_date, 
           SUM(dock_count) AS total_dock_ct,
           COUNT(name) AS station_count
    FROM station
    GROUP BY 1,2
    ORDER BY 2,1;
''', con=conn)

,city,installed_date,total_dock_ct,station_count
0,San Jose,2013-08-05,83,5
1,San Jose,2013-08-06,87,5
2,San Jose,2013-08-07,64,4
3,Redwood City,2013-08-12,60,4
4,Palo Alto,2013-08-14,64,4
5,Mountain View,2013-08-15,46,2
6,Palo Alto,2013-08-15,11,1
7,Redwood City,2013-08-15,40,2
8,Mountain View,2013-08-16,45,3
9,San Francisco,2013-08-19,94,6


In [7]:
#Installment 
pd.read_sql('''
    WITH t1 AS (SELECT city,
                       CASE
                       -- m/d/yyyy
                       WHEN (length(installation_date) = 8 AND substr(installation_date,2,1) = '/') 
                       THEN substr(installation_date,5,4)||'-0'||substr(installation_date,1,1)||'-0'||substr(installation_date,3,1)
                       -- m/dd/yyyy
                       WHEN (length(installation_date) = 9 AND substr(installation_date,2,1) = '/') 
                       THEN substr(installation_date,6,4)||'-0'||substr(installation_date,1,1)||'-'||substr(installation_date,3,2)
                       -- mm/d/yyyy
                       WHEN (length(installation_date) = 9 AND substr(installation_date,3,1) = '/') 
                       THEN substr(installation_date,6,4)||'-0'||substr(installation_date,1,2)||'-'||substr(installation_date,4,1)
                       -- mm/dd/yyyy
                       WHEN (length(installation_date) = 10 AND substr(installation_date,3,1) = '/') 
                       THEN substr(installation_date,7,4)||'-'||substr(installation_date,1,2)||'-'||substr(installation_date,4,2)
                       ELSE installation_date
                       END AS installed_date, 
                       SUM(dock_count) AS total_dock_ct,
                       COUNT(name) AS station_count
                FROM station
                GROUP BY 1,2
                ORDER BY 2,1)
    SELECT CASE 
        WHEN month = '2013-08-01'
        THEN 'AUG13'
        WHEN month > '2013-08-01'
        THEN 'after_AUG13'
        ELSE 'before_AUG13'
        END AS installation_month,
        SUM(total_dock_ct) AS dock_ct,
        SUM(station_count) AS station_ct
    FROM (SELECT DATE(installed_date, 'start of month') as month,
                 total_dock_ct, 
                 station_count
         FROM t1
         ) AS innerquery
    GROUP BY 1;
''', con=conn, index_col='installation_month')

,dock_ct,station_ct
installation_month,,
AUG13,1150,64
after_AUG13,86,6


In [8]:
pd.read_sql('''
     SELECT * 
     FROM status
     LIMIT 10;
''', con=conn)

,station_id,bikes_available,docks_available,time
0,2,2,25,2013/08/29 12:06:01
1,2,2,25,2013/08/29 12:07:01
2,2,2,25,2013/08/29 12:08:01
3,2,2,25,2013/08/29 12:09:01
4,2,2,25,2013/08/29 12:10:01
5,2,2,25,2013/08/29 12:11:01
6,2,2,25,2013/08/29 12:12:01
7,2,2,25,2013/08/29 12:13:01
8,2,2,25,2013/08/29 12:15:01
9,2,2,25,2013/08/29 12:16:02


In [9]:
#Bikes availablity vs docks
pd.read_sql('''
     SELECT ROUND(AVG(status.bikes_available),2) AS avg_available_bikes, 
            ROUND(AVG(status.docks_available),2) AS free_dock_count,
            station.dock_count AS max_dock_capacity,
            station.name
     FROM status
     INNER JOIN station
     ON status.station_id = station.id
     GROUP BY name
     ORDER BY 1, 2 DESC
     LIMIT 10;
''', con=conn)

,avg_available_bikes,free_dock_count,max_dock_capacity,name
0,4.73,6.26,11,Castro Street and El Camino Real
1,5.27,5.72,11,Cowper at University
2,5.29,5.69,11,Santa Clara at Almaden
3,5.67,9.32,15,Commercial at Montgomery
4,6.09,8.89,15,Broadway St at Battery St
5,6.13,12.81,19,2nd at Folsom
6,6.13,8.84,15,Clay at Battery
7,6.22,4.76,11,University and Emerson
8,6.25,8.73,15,Embarcadero at Vallejo
9,6.36,8.63,15,San Jose City Hall


In [23]:
pd.read_sql('''
    SELECT ROUND(AVG(status.bikes_available),2) AS avg_available_bikes, 
           ROUND(AVG(status.docks_available),2) AS free_dock_count,
           COUNT(*) AS num_occurrences,
           DATE(SUBSTR(time, 1,4) || '-' || SUBSTR(time, 6,2) || '-' || 
                SUBSTR(time, 9,2) ||  SUBSTR(time, 11,9), 'start of month') AS month 
    FROM status
    INNER JOIN station
    ON status.station_id = station.id
    WHERE station.name = '2nd at Folsom'
    GROUP BY station.name, month
    ORDER BY 2 DESC;
''', con=conn)

,avg_available_bikes,free_dock_count,num_occurrences,month
0,4.81,14.19,3245,2013-08-01
1,5.13,13.84,41158,2014-04-01
2,5.19,13.81,41982,2014-09-01
3,5.38,13.59,42918,2015-06-01
4,5.49,13.47,44629,2015-05-01
5,5.35,13.46,43046,2015-04-01
6,5.65,13.35,44639,2014-05-01
7,5.72,13.27,43998,2013-10-01
8,5.77,13.23,43194,2014-11-01
9,5.61,13.07,44578,2015-03-01


In [11]:
pd.read_sql('''
    SELECT *
    FROM trip
    LIMIT 10
''', con=conn)

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4069,174,8/29/2013 9:08,2nd at South Park,64,8/29/2013 9:11,2nd at South Park,64,288,Subscriber,94114
1,4073,1067,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:42,San Francisco Caltrain 2 (330 Townsend),69,321,Subscriber,94703
2,4074,1131,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,317,Subscriber,94115
3,4075,1117,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,316,Subscriber,94122
4,4076,1118,8/29/2013 9:25,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,322,Subscriber,94597
5,4078,3829,8/29/2013 9:31,Redwood City Caltrain Station,22,8/29/2013 10:34,Redwood City Caltrain Station,22,228,Customer,94062
6,4079,995,8/29/2013 9:35,South Van Ness at Market,66,8/29/2013 9:52,South Van Ness at Market,66,327,Subscriber,94102
7,4080,764,8/29/2013 9:36,South Van Ness at Market,66,8/29/2013 9:49,San Francisco Caltrain 2 (330 Townsend),69,315,Subscriber,94117
8,4081,218,8/29/2013 9:38,Mountain View City Hall,27,8/29/2013 9:41,Mountain View City Hall,27,150,Subscriber,97214
9,4084,287,8/29/2013 9:41,Mountain View City Hall,27,8/29/2013 9:46,Mountain View City Hall,27,138,Customer,97214


In [12]:
# Trip Duration Analysis
pd.read_sql('''
    SELECT duration/60/60 AS duration_hr, COUNT(*) AS frequency
    FROM trip
    GROUP BY duration_hr
    ORDER BY duration_hr DESC 
    LIMIT 15;
''', con=conn)

,duration_hr,frequency
0,4797,1
1,593,1
2,514,1
3,314,1
4,200,2
5,199,1
6,198,1
7,191,1
8,182,1
9,179,1


In [13]:
pd.read_sql('''
    SELECT duration/60/60 AS duration_hr, COUNT(*) AS frequency
    FROM trip
    WHERE duration_hr <= 15
    GROUP BY duration_hr
    ORDER BY duration_hr DESC;
''', con=conn)

,duration_hr,frequency
0,15,61
1,14,75
2,13,64
3,12,72
4,11,99
5,10,89
6,9,149
7,8,368
8,7,473
9,6,728


In [14]:
#Busiest pickup and dropoff locations
pd.read_sql('''
    SELECT COUNT(*) AS num_count,
           trip.start_station_name
    FROM trip
    INNER JOIN station
    ON station.id = start_station_id
    WHERE station.city = 'San Francisco'
    GROUP BY 2
    ORDER BY 1 DESC
    LIMIT 5;
''', con=conn)

,num_count,start_station_name
0,49092,San Francisco Caltrain (Townsend at 4th)
1,33742,San Francisco Caltrain 2 (330 Townsend)
2,32934,Harry Bridges Plaza (Ferry Building)
3,27713,Embarcadero at Sansome
4,26089,Temporary Transbay Terminal (Howard at Beale)


In [15]:
pd.read_sql('''
    SELECT COUNT(*) AS num_count, 
           trip.end_station_name
    FROM trip
    INNER JOIN station
    ON station.id = start_station_id
    WHERE station.city = 'San Francisco'
    GROUP BY 2
    ORDER BY 1 DESC
    LIMIT 5;
''', con=conn)

,num_count,end_station_name
0,63167,San Francisco Caltrain (Townsend at 4th)
1,35117,San Francisco Caltrain 2 (330 Townsend)
2,33193,Harry Bridges Plaza (Ferry Building)
3,30796,Embarcadero at Sansome
4,28529,2nd at Townsend


In [16]:
pd.read_sql('''
    SELECT COUNT(*) AS num_count,
           ROUND(AVG(duration/60), 2) AS avg_duration_mins,
           trip.start_station_name,
           trip.end_station_name
    FROM trip
    INNER JOIN station
    ON station.id = start_station_id
    WHERE station.city = 'San Francisco' AND duration/60/60 <= 15
    GROUP BY 3, 4
    ORDER BY 1 DESC
    LIMIT 15;
''', con=conn)

,num_count,avg_duration_mins,start_station_name,end_station_name
0,6215,4.55,San Francisco Caltrain 2 (330 Townsend),Townsend at 7th
1,6163,20.39,Harry Bridges Plaza (Ferry Building),Embarcadero at Sansome
2,5041,4.26,Townsend at 7th,San Francisco Caltrain (Townsend at 4th)
3,4838,8.98,2nd at Townsend,Harry Bridges Plaza (Ferry Building)
4,4357,10.65,Harry Bridges Plaza (Ferry Building),2nd at Townsend
5,4269,8.54,Embarcadero at Sansome,Steuart at Market
6,3963,10.24,Embarcadero at Folsom,San Francisco Caltrain (Townsend at 4th)
7,3902,8.84,Steuart at Market,2nd at Townsend
8,3627,8.51,2nd at South Park,Market at Sansome
9,3621,13.21,San Francisco Caltrain (Townsend at 4th),Harry Bridges Plaza (Ferry Building)


In [17]:
pd.read_sql('''
    SELECT AVG(avg_duration_mins) AS avg_duration_for_most_pop
    FROM (SELECT COUNT(*) AS num_count,
                 ROUND(AVG(duration/60), 2) AS avg_duration_mins,
                 trip.start_station_name,
                 trip.end_station_name
          FROM trip
          INNER JOIN station
          ON station.id = start_station_id
          WHERE station.city = 'San Francisco' AND duration/60/60 <= 15
          GROUP BY 3, 4
          HAVING COUNT(*) > 2000
          ORDER BY 1 DESC) AS most_popular_trips
''', con=conn)

,avg_duration_for_most_pop
0,10.735306


In [18]:
#Popular long-trip routes 
pd.read_sql('''
    SELECT COUNT(*) AS num_count,
           ROUND(AVG(duration/60), 2) AS avg_duration_mins,
           trip.start_station_name,
           trip.end_station_name
    FROM trip
    INNER JOIN station
    ON station.id = start_station_id
    WHERE city='San Francisco' AND duration/60/60 <= 15
    GROUP BY 3, 4
    HAVING COUNT(*) > 50
    ORDER BY 2 DESC
    LIMIT 15;
''', con=conn)

,num_count,avg_duration_mins,start_station_name,end_station_name
0,605,145.30,Powell at Post (Union Square),Powell at Post (Union Square)
1,88,121.31,Washington at Kearney,Washington at Kearney
2,600,114.90,Powell Street BART,Powell Street BART
3,715,114.75,Market at 4th,Market at 4th
4,254,109.41,Beale at Market,Beale at Market
5,476,104.71,Grant Avenue at Columbus Avenue,Grant Avenue at Columbus Avenue
6,426,103.59,Civic Center BART (7th at Market),Civic Center BART (7th at Market)
7,146,101.99,Post at Kearny,Post at Kearny
8,230,100.49,Washington at Kearny,Washington at Kearny
9,206,97.50,Golden Gate at Polk,Golden Gate at Polk


In [19]:
pd.read_sql('''
    SELECT subscription_type,
           COUNT(*) AS sub_type_ct
    FROM trip
    GROUP BY 1;
''', con=conn)

,subscription_type,sub_type_ct
0,Customer,103213
1,Subscriber,566746


In [20]:
#Average duration per subscriber type
pd.read_sql('''
    WITH t1 AS (SELECT DATE(CASE
                -- m/d/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 8 AND substr(start_date,2,1) = '/') 
               THEN substr(start_date,5,4)||'-0'||substr(start_date,1,1)||'-0'||substr(start_date,3,1)
               -- m/dd/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 9 AND substr(start_date,2,1) = '/') 
               THEN substr(start_date,6,4)||'-0'||substr(start_date,1,1)||'-'||substr(start_date,3,2)
               -- mm/d/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 9 AND substr(start_date,3,1) = '/') 
               THEN substr(start_date,6,4)||'-0'||substr(start_date,1,2)||'-'||substr(start_date,4,1)
               -- mm/dd/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 10 AND substr(start_date,3,1) = '/') 
               THEN substr(start_date,7,4)||'-'||substr(start_date,1,2)||'-'||substr(start_date,4,2)
               ELSE start_date
               END) AS trip_date,
               subscription_type, 
               (duration / 60) AS duration_min
        FROM trip
        INNER JOIN station
        ON station.id = start_station_id
        WHERE city='San Francisco' AND duration/60/60 <= 15)
    SELECT CASE 
           WHEN (trip_date IN (DATE(trip_date, 'weekday 6'), DATE(trip_date, 'weekday 0')))
           THEN 'weekends'
           ELSE 'weekdays'
           END AS weekday,
           subscription_type,
           ROUND(AVG(duration_min), 2) AS avg_dur_min
           
    FROM t1
    GROUP BY 1,2;
''', con=conn)

,weekday,subscription_type,avg_dur_min
0,weekdays,Customer,42.46
1,weekdays,Subscriber,8.90
2,weekends,Customer,53.27
3,weekends,Subscriber,10.04


In [21]:
pd.read_sql('''
    WITH t1 AS (SELECT DATE(CASE
                -- m/d/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 8 AND substr(start_date,2,1) = '/') 
               THEN substr(start_date,5,4)||'-0'||substr(start_date,1,1)||'-0'||substr(start_date,3,1)
               -- m/dd/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 9 AND substr(start_date,2,1) = '/') 
               THEN substr(start_date,6,4)||'-0'||substr(start_date,1,1)||'-'||substr(start_date,3,2)
               -- mm/d/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 9 AND substr(start_date,3,1) = '/') 
               THEN substr(start_date,6,4)||'-0'||substr(start_date,1,2)||'-'||substr(start_date,4,1)
               -- mm/dd/yyyy
               WHEN ((INSTR(start_date, ' ')-1) = 10 AND substr(start_date,3,1) = '/') 
               THEN substr(start_date,7,4)||'-'||substr(start_date,1,2)||'-'||substr(start_date,4,2)
               ELSE start_date
               END) AS trip_date,
               subscription_type, 
               (duration / 60) AS duration_min
        FROM trip
        INNER JOIN station
        ON station.id = start_station_id
        WHERE city='San Francisco' AND duration/60/60 <= 15)
    SELECT CASE 
           WHEN trip_date = (DATE(trip_date, 'weekday 1'))
           THEN '1 - Monday'
           WHEN trip_date = (DATE(trip_date, 'weekday 2'))
           THEN '2 - Tuesday'
           WHEN trip_date = (DATE(trip_date, 'weekday 3'))
           THEN '3 - Wednesday'
           WHEN trip_date = (DATE(trip_date, 'weekday 4'))
           THEN '4 - Thursday'
           WHEN trip_date = (DATE(trip_date, 'weekday 5'))
           THEN '5 - Friday'
           WHEN trip_date = (DATE(trip_date, 'weekday 6'))
           THEN '6 - Saturday'
           ELSE '7 - Sunday'
           END AS weekday,
           subscription_type,
           ROUND(AVG(duration_min), 2) AS avg_dur_min
           
    FROM t1
    GROUP BY 1,2
    ORDER BY 2,1;
''', con=conn)

,weekday,subscription_type,avg_dur_min
0,1 - Monday,Customer,43.85
1,2 - Tuesday,Customer,42.43
2,3 - Wednesday,Customer,41.78
3,4 - Thursday,Customer,40.91
4,5 - Friday,Customer,43.22
5,6 - Saturday,Customer,52.29
6,7 - Sunday,Customer,50.92
7,1 - Monday,Subscriber,8.83
8,2 - Tuesday,Subscriber,8.86
9,3 - Wednesday,Subscriber,9.01


In [22]:
#Popular locations and average duration time
pd.read_sql('''
    SELECT COUNT(*) AS num_count,
           ROUND(AVG(duration/60), 2) AS avg_duration_mins,
           trip.start_station_name,
           trip.end_station_name
    FROM trip
    INNER JOIN station
    ON station.id = start_station_id
    WHERE station.city = 'San Francisco' AND duration/60/60 <= 15 AND subscription_type = 'Customer'
    GROUP BY 3, 4
    ORDER BY 1 DESC
    LIMIT 15;
''', con=conn)

,num_count,avg_duration_mins,start_station_name,end_station_name
0,2708,37.04,Harry Bridges Plaza (Ferry Building),Embarcadero at Sansome
1,1837,75.31,Embarcadero at Sansome,Embarcadero at Sansome
2,1482,108.12,Harry Bridges Plaza (Ferry Building),Harry Bridges Plaza (Ferry Building)
3,1237,26.03,Embarcadero at Sansome,Harry Bridges Plaza (Ferry Building)
4,1005,41.79,Embarcadero at Vallejo,Embarcadero at Sansome
5,649,52.22,Harry Bridges Plaza (Ferry Building),Embarcadero at Vallejo
6,629,34.33,Steuart at Market,Embarcadero at Sansome
7,599,133.75,Market at 4th,Market at 4th
8,569,34.42,Embarcadero at Sansome,Market at 4th
9,565,49.85,Market at 4th,Embarcadero at Sansome
